In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/30-days-of-ml/sample_submission.csv
/kaggle/input/30-days-of-ml/train.csv
/kaggle/input/30-days-of-ml/test.csv


In [2]:
from sklearn import model_selection
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
test = pd.read_csv('/kaggle/input/30-days-of-ml/test.csv')
train= pd.read_csv('/kaggle/input/30-days-of-ml/train.csv')

In [4]:
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

In [5]:
# let's use kfold for cros validation 

kf = model_selection.KFold(n_splits = 5, shuffle = True , random_state = 5)
for fold, (train_indices, valid_indices) in enumerate (kf.split(X = train)):
    train.loc[valid_indices, 'kfold'] = fold

In [6]:
df_train = train

In [7]:
features = [c for c in df_train.columns if c not in ('id', 'kfold', 'target')]
obj_cols = [col for col in features if 'cat' in col]
test = test[features]

In [8]:
final_pred = []
for fold in range(5):
    xtrain = df_train[df_train.kfold != fold].reset_index(drop =True)
    xvalid = df_train[df_train.kfold == fold].reset_index(drop =True)
    xtest = test.copy()
    
    ytrain = xtrain.target
    yvalid = xvalid.target
   
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    
    print('encoding')
    encoder = OrdinalEncoder()
    xtrain[obj_cols] = encoder.fit_transform(xtrain[obj_cols])
    xvalid[obj_cols] = encoder.fit_transform(xvalid[obj_cols])
    xtest[obj_cols] = encoder.fit_transform(xtest[obj_cols])
    
    print('training')
    model = RandomForestRegressor(random_state=fold, n_jobs = -1)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_pred = model.predict(xtest)
    final_pred.append(test_pred)
    print(fold, mean_squared_error(yvalid, preds_valid, squared=False))
    

encoding
training
0 0.7348066097782634
encoding
training
1 0.7358421244502356
encoding
training
2 0.7370368869620207
encoding
training
3 0.7327045732923444
encoding
training
4 0.7377941054771865


In [9]:
preds = np.mean(np.column_stack(final_pred), axis=1)
sample_submission.target = preds
sample_submission.to_csv("submission.csv", index=False)